In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [2]:
%matplotlib inline

In [3]:
customers = pd.read_csv("datasets/customers.csv")
loyalty = pd.read_csv("datasets/loyalty_members.csv")
orders = pd.read_csv("datasets/orders.csv")
order_details = pd.read_csv("datasets/order_details.csv")

food_menu = pd.read_csv("datasets/food_menu.csv")
drinks_menu = pd.read_csv("datasets/drinks_menu.csv")
games_menu = pd.read_csv("datasets/games_menu.csv")

In [4]:
# head, shape of each dataframe
dfs = {
    "customers": customers,
    "loyalty": loyalty,
    "orders": orders,
    "order_details": order_details,
    "food_menu": food_menu,
    "drinks_menu": drinks_menu,
    "games_menu": games_menu,
}

for name, df in dfs.items():
    print(f"\n===== {name.upper()} =====")
    print(df.head(3))
    print(df.shape)


===== CUSTOMERS =====
              name                       email           phone  \
0  Mason MacDonald  mason.macdonald0@gmail.com  1-365-227-6804   
1      Noah Wilson      noah.wilson1@yahoo.com  1-647-501-6525   
2       Nora Smith       nora.smith2@yahoo.com  1-365-422-7133   

  loyalty_enrolled  
0                N  
1                Y  
2                N  
(500, 4)

===== LOYALTY =====
  loyalty_id                       email      tier  points_earned  gender  \
0    L000002      noah.wilson1@yahoo.com  Frequent           6415    Male   
1    L000005  layla.gonzalez4@icloud.com  Frequent           5245  Female   
2    L000009       mila.nowak8@yahoo.com  Frequent           7047    Male   

   age    birthday         occupation            name  
0   28  1997-03-18        Electrician     Noah Wilson  
1   33  1992-02-18  Software Engineer  Layla Gonzalez  
2   25  2000-08-13   Graphic Designer      Mila Nowak  
(308, 9)

===== ORDERS =====
  transaction_id                    

# Merge all menu -> menu_all

In [5]:
food_menu["item_type"] = "food"
drinks_menu["item_type"] = "drink"
games_menu["item_type"] = "entertainment"

menu_all = pd.concat([food_menu, drinks_menu, games_menu], ignore_index=True)

menu_all

,id,name,category,unit_price_cad,item_type
0,F001,Classic Burger,Mains,17.99,food
1,F002,Chicken Wings,Appetizers,15.49,food
2,F003,Nachos Supreme,Appetizers,16.99,food
3,F004,Mac & Cheese,Mains,14.49,food
4,F005,BBQ Ribs Half Rack,Mains,23.99,food
5,F006,Caesar Salad,Salads,12.49,food
6,F007,Veggie Pizza,Mains,18.99,food
7,F008,Steak Sandwich,Mains,21.49,food
8,F009,Fish Tacos,Mains,16.49,food
9,F010,Chocolate Lava Cake,Desserts,8.99,food


In [6]:
# Add columns to customers: visit_count, total_spent
visit_count_df = orders.groupby("email")["transaction_id"].nunique().reset_index(name="visit_count")
total_spent_df = orders.groupby("email")["total_bill_amount"].sum().reset_index(name="total_spent")
customers = customers.merge(visit_count_df, on="email", how="left") \
                     .merge(total_spent_df, on="email", how="left")

#drop phone number and name
customers = customers.drop(columns=["phone","name"])


customers.head(10)


,email,loyalty_enrolled,visit_count,total_spent
0,mason.macdonald0@gmail.com,N,7,763.26
1,noah.wilson1@yahoo.com,Y,6,641.59
2,nora.smith2@yahoo.com,N,11,1386.86
3,jackson.liu3@outlook.com,N,7,441.27
4,layla.gonzalez4@icloud.com,Y,5,524.54
5,harper.nowak5@outlook.com,N,7,292.39
6,carter.patel6@yahoo.com,N,8,718.99
7,isabella.tremblay7@outlook.com,N,7,551.16
8,mila.nowak8@yahoo.com,Y,8,704.71
9,alexander.lau9@hotmail.com,Y,12,876.98
